In [1]:
import numpy as np

In [19]:
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats

In [23]:
round(stats.norm.ppf(1 - 0.003 / 2), 4)

2.9677

In [25]:
round(189 / 11034, 4)

0.0171

In [26]:
round(104 / 11037, 4)

0.0094

In [27]:
0.0171 - 0.0094

0.0077

In [3]:
placebo = (189/11034) / (1 - (189/11034))
aspirin = (104/11037) / (1 - (104/11037))
round(placebo/aspirin, 4)

1.8321

In [4]:
round(aspirin/placebo, 4)

0.5458

In [5]:
from statsmodels.stats.proportion import proportion_confint

In [6]:
normal_interval = proportion_confint(104, 11037, method='normal')

In [7]:
print('normal_interval [%f, %f] with width %f' % (normal_interval[0],
                                                  normal_interval[1], 
                                                  normal_interval[1] - normal_interval[0]))

normal_interval [0.007620, 0.011225] with width 0.003605


In [8]:
from scipy import stats as st

In [9]:
def proportion_confint_2(count, nobs, alpha=0.05, method='normal'):
    q_ = count * 1 / nobs
    alpha_2 = 0.5 * alpha
    std_ = np.sqrt(q_ * (1 - q_) / nobs)
    dist = st.norm.isf(alpha / 2) * std_
    ci_low = q_ - dist
    ci_upp = q_ + dist
    return round(ci_low, 4), round(ci_upp, 4)

In [10]:
proportion_confint_2(189, 11034)

(0.0147, 0.0195)

In [11]:
proportion_confint_2(104, 11037)

(0.0076, 0.0112)

In [31]:
aspirin_data = np.array([1 if i<104 else 0 for i in range(11037)])
placebo_data = np.array([1 if i<189 else 0 for i in range(11034)])

In [35]:
def proportions_confint_diff_ind(sample1, sample2, alpha=0.05):    
    z = stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (round(left_boundary, 4), round(right_boundary, 4))

In [36]:
proportions_confint_diff_ind(placebo_data, aspirin_data)

(0.0047, 0.0107)

In [14]:
vect_asp = [1 for i in range(104)] + [0 for i in range(11037)]
vect_plac = [1 for i in range(189)] + [0 for i in range(11034)]

In [15]:
vect_a = np.array(vect_asp)
vect_p = np.array(vect_plac)

In [38]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [39]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [44]:
def odds(data):
    p = data.sum() / data.shape[0]
    return p / (1 - p)

In [48]:
# np.random.seed(seed=0)
odds_aspirin_data = np.array(list(map(odds, get_bootstrap_samples(aspirin_data, 1000))))
odds_placebo_data = np.array(list(map(odds, get_bootstrap_samples(placebo_data, 1000))))
stat_intervals(odds_placebo_data / odds_aspirin_data, 0.05)

array([1.46347962, 2.40055445])